In [2]:
import ots_backend as backend
from json import loads
import pandas as pd
from requests import get

In [3]:
ncaaf = pd.concat([pd.read_csv('../database/ncaaf_2023.csv'), pd.read_csv('../database/ncaaf_2023.csv')])

In [4]:
ncaaf['date'] = None
ncaaf['venue'] = None
ncaaf['stadium_indoor'] = None
ncaaf['city'] = None
ncaaf['state'] = None
ncaaf['zip'] = None
ncaaf['grass'] = None
ncaaf['attendance'] = None

ncaaf['away_id'] = None
ncaaf['away_school'] = None
ncaaf['away_name'] = None
ncaaf['away_abbreviation'] = None
ncaaf['away_color'] = None

ncaaf['home_id'] = None
ncaaf['home_school'] = None
ncaaf['home_name'] = None
ncaaf['home_abbreviation'] = None
ncaaf['home_color'] = None
ncaaf['neutral_site'] = None

for i in range(len(ncaaf)):
    extract_main = loads(ncaaf['extract_main'].values[i])
    extract_from_url = loads(ncaaf['extract_from_url'].values[i])
    
    ncaaf['date'].values[i] = extract_main.get('header').get('competitions')[0].get('date')
    ncaaf['venue'].values[i] = extract_main.get('gameInfo').get('venue').get('fullName')
    ncaaf['stadium_indoor'].values[i]  = extract_from_url.get('competitions')[0].get('venue').get('indoor')
    ncaaf['city'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('city')
    ncaaf['state'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('state')
    ncaaf['zip'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('zipCode')
    ncaaf['grass'].values[i] = extract_main.get('gameInfo').get('venue').get('grass')
    ncaaf['attendance'].values[i] = extract_main.get('gameInfo').get('attendance')

    ncaaf['away_id'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('id', None)
    ncaaf['away_school'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('location', None)
    ncaaf['away_name'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('name', None)
    ncaaf['away_abbreviation'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('abbreviation', None)
    ncaaf['away_color'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('color', None)

    ncaaf['home_id'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('id', None)
    ncaaf['home_school'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('location', None)
    ncaaf['home_name'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('name', None)
    ncaaf['home_abbreviation'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('abbreviation', None)
    ncaaf['home_color'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('color', None)

    ncaaf['neutral_site'].values[i] = extract_from_url.get('competitions')[0].get('neutralSite')

Teams = ncaaf[['home_id', 'home_school', 'home_name', 'home_abbreviation', 'home_color', 'venue', 'city', 'state', 'zip', 'grass', 'stadium_indoor', 'neutral_site']]
Teams = Teams.rename(columns = {'home_id':'espn_id', 'home_school':'school', 'home_name':'name', 'home_abbreviation':'abbreviation', 'home_color':'color', 'venue':'stadium'})
Teams_count = Teams.groupby('school').count().sort_values('espn_id').reset_index(drop=False)
Teams_count = Teams_count.rename(columns = {'espn_id':'espn_id_count'})
Teams_count = Teams_count[['school', 'espn_id_count']]
Teams = Teams.merge(Teams_count, how = 'left', on = 'school')
Teams = Teams[Teams['espn_id_count'] > 5]
Teams = Teams[Teams['neutral_site'] == False]
Teams = Teams.drop_duplicates(subset=['school'])
Teams = Teams.drop(columns=['espn_id_count', 'neutral_site'])
Teams['division'] = None

In [5]:
for i in range(len(Teams)):
    espn_id = Teams['espn_id'].values[i]
    res = get('https://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/teams/' + str(espn_id) + '?lang=en&region=us')
    res = loads(res.content.decode('utf-8'))
    res = get(res.get('groups').get('$ref'))
    res = loads(res.content.decode('utf-8'))
    
    Teams['division'].values[i] = res.get('midsizeName')

In [6]:
Teams.to_csv('../ots_backend/assets/ncaaf_teams.csv', index=False)

In [8]:
Teams.to_json('../ots_backend/assets/ncaaf_teams.json', index=False, orient='records')